<a href="https://colab.research.google.com/github/kainat5008/Assignment-3/blob/main/ML_Assignment_Task5_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask pyngrok joblib numpy huggingface_hub

In [ ]:
!pip install flask_ngrok

In [10]:
!pip install flask pyngrok joblib numpy huggingface_hub
from flask import Flask, request, jsonify, render_template_string
import joblib
import numpy as np
from huggingface_hub import hf_hub_download
from pyngrok import ngrok
from google.colab import userdata
import os

# Set up authentication tokens
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')  # From Colab secrets
ngrok.set_auth_token("2vffXqpQXoZ23eS7GhOEVqjXDez_NTwqaFkXepGEVgK6E7v9")  # Your ngrok token

# HTML Template with improved styling
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Housing Price Predictor</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 600px;
            margin: 0 auto;
            padding: 20px;
            line-height: 1.6;
        }
        h1 {
            color: #2c3e50;
            text-align: center;
        }
        .input-group {
            margin-bottom: 15px;
        }
        input {
            width: 100%;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 4px;
            box-sizing: border-box;
        }
        button {
            background: #3498db;
            color: white;
            border: none;
            padding: 10px 15px;
            border-radius: 4px;
            cursor: pointer;
            width: 100%;
            font-size: 16px;
        }
        button:hover {
            background: #2980b9;
        }
        .error {
            color: #e74c3c;
            margin-top: 10px;
        }
        .result {
            margin-top: 20px;
            padding: 15px;
            background: #f8f9fa;
            border-radius: 4px;
            font-size: 18px;
            text-align: center;
        }
        .example {
            font-size: 14px;
            color: #7f8c8d;
            margin-top: 5px;
        }
    </style>
</head>
<body>
    <h1>California Housing Price Predictor</h1>
    <form action="/predict" method="post">
        <div class="input-group">
            <label for="features">Enter 8 space-separated values:</label>
            <input type="text" id="features" name="features"
                   placeholder="0.1 20.0 5.0 1.0 500.0 6.0 40.0 -122.0" required>
            <div class="example">Example: 0.1 20.0 5.0 1.0 500.0 6.0 40.0 -122.0</div>
        </div>
        <button type="submit">Predict Price</button>
    </form>
    {% if error %}
        <div class="error">{{ error }}</div>
    {% endif %}
    {% if prediction %}
        <div class="result">Predicted Price: ${{ prediction }}</div>
    {% endif %}
</body>
</html>
"""

app = Flask(__name__)

# Load model coefficients and scaler with enhanced error handling
try:
    print("Loading model and scaler from Hugging Face Hub...")
    model_path = hf_hub_download(
        repo_id="keenu-5008/california-housing-regression",
        filename="batch_gd_model.pkl",
        token=os.environ.get("HF_TOKEN")
    )
    scaler_path = hf_hub_download(
        repo_id="keenu-5008/california-housing-regression",
        filename="scaler.pkl",
        token=os.environ.get("HF_TOKEN")
    )

    # Load the files
    model_coefficients = joblib.load(model_path)
    scaler = joblib.load(scaler_path)

    # Verify loaded objects
    if not isinstance(model_coefficients, np.ndarray):
        raise ValueError("Model file should contain numpy array of coefficients")
    if not hasattr(scaler, 'transform'):
        raise ValueError("Scaler object is invalid")

    print(f"Successfully loaded model coefficients (shape: {model_coefficients.shape})")
    print(f"Scaler type: {type(scaler)}")

except Exception as e:
    print(f"\n❌ Error loading model: {str(e)}\n")
    raise

def make_prediction(features):
    """Helper function to make predictions using the coefficients"""
    try:
        # Convert input to numpy array
        input_data = np.array(features).reshape(1, -1)

        # Scale the features
        scaled_data = scaler.transform(input_data)

        # Add bias term (intercept)
        scaled_data_with_bias = np.c_[np.ones(scaled_data.shape[0]), scaled_data]

        # Manual prediction using dot product (since we have coefficients)
        prediction = np.dot(scaled_data_with_bias, model_coefficients)[0]

        return prediction * 1000  # Convert to dollar value

    except Exception as e:
        print(f"Prediction error: {str(e)}")
        raise

@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get and validate input
        features = request.form['features'].strip()
        if not features:
            return render_template_string(
                HTML_TEMPLATE,
                error="Please enter some values",
                prediction=None
            )

        # Split and validate features
        feature_values = features.split()
        if len(feature_values) != 8:
            return render_template_string(
                HTML_TEMPLATE,
                error="Please enter exactly 8 numbers separated by spaces",
                prediction=None
            )

        # Convert to floats
        try:
            float_features = [float(x) for x in feature_values]
        except ValueError:
            return render_template_string(
                HTML_TEMPLATE,
                error="All values must be numbers",
                prediction=None
            )

        # Make prediction
        prediction = make_prediction(float_features)

        # Format the prediction as currency
        formatted_prediction = "{:,.2f}".format(prediction)

        return render_template_string(
            HTML_TEMPLATE,
            prediction=formatted_prediction,
            error=None
        )

    except Exception as e:
        error_msg = f"Prediction failed: {str(e)}"
        print(error_msg)
        return render_template_string(
            HTML_TEMPLATE,
            error=error_msg,
            prediction=None
        )

# Start the app
if __name__ == '__main__':
    try:
        public_url = ngrok.connect(5000)
        print(f"\n⭐ App is running at: {public_url}\n")
        print("Note: The ngrok URL will expire after 2 hours")
        app.run()
    except Exception as e:
        print(f"\n❌ Failed to start app: {str(e)}\n")

Loading model and scaler from Hugging Face Hub...
Successfully loaded model coefficients (shape: (9,))
Scaler type: <class 'sklearn.preprocessing._data.StandardScaler'>

⭐ App is running at: NgrokTunnel: "https://4ef0-35-233-250-200.ngrok-free.app" -> "http://localhost:5000"

Note: The ngrok URL will expire after 2 hours
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [13/Apr/2025 11:37:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Apr/2025 11:37:30] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Apr/2025 11:37:34] "POST /predict HTTP/1.1" 200 -
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise Run